In [1]:

import matplotlib.pyplot as plt
#import scipy.stats as sts
import numpy as np
import cmdstanpy ## import stan interface for Python
#from scipy.integrate import solve_ivp
#from matplotlib.gridspec import GridSpec
import os
import seaborn as sns
import pandas as pd
#import math
# from scipy.special import logit,expit
from importlib import reload 
import sys
# import patchworklib as pw
sys.path.append("..")
from statannotations.Annotator import Annotator
from itertools import product
import matplotlib.ticker as mticker
import arviz
import matplotlib as mpl

import warnings
warnings.filterwarnings("ignore")

tex_fonts = {
    # Use LaTeX to write all text
#     "text.usetex": True,
    "font.family": "Helvetica",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 7,
    "font.size": 7,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 6,
    "xtick.labelsize": 6,
    "ytick.labelsize": 6,
    "axes.grid": True,
    'grid.color': '#DDDDDD',
    'grid.linestyle': '-',
    'grid.linewidth': 0.3,
    "lines.markersize":4,
    
    'axes.axisbelow':True,
    'lines.linewidth':1,
    'pdf.fonttype':42,
    'axes.linewidth':0.6,
    'xtick.major.width':0.5,
    'ytick.major.width':0.5,
    'ytick.minor.width':0.3,

    'ytick.major.pad':0.2,
        "xtick.major.size":3,
    "ytick.major.size":3,
}

plt.rcParams.update(tex_fonts)



# label_size = 30
# mpl.rcParams['xtick.labelsize'] = label_size 
# mpl.rcParams['ytick.labelsize'] = label_size 
# mpl.rcParams['lines.markersize'] = 20  
# plt.rcParams['axes.grid'] = True
# mpl.rcParams['font.family']='Helvetica'

ratio= 2/(1+np.sqrt(5))

width = 520.344
if os.name == "nt": ## adds compiler to path in Windows
    cmdstanpy.utils.cxx_toolchain_path() 
    
# %config InlineBackend.close_figures=False # keep figures open in pyplot
# #%config InlineBackend.print_figure_kwargs = {'bbox_inches':"tight", 'pad_inches':2}


# sigma = 0.15

# switchtime = 21
# numObs = 27
# tmax = 35

# # time_index = np.concatenate((np.logspace(-1,np.log10(switchtime),num=int(numObs/2)), (np.logspace(-1,np.log10(tmax-switchtime),num=int(numObs/2)))+switchtime))
# #time_index = np.linspace(tmax/numObs,tmax,numObs)
# switch = (numofbrduint+2)*(numofki67int+2)
# y0_est = np.zeros((numofki67int+2)*(numofbrduint+2)*2)

time_index_equilibrium = np.linspace(10000/1, 10000, 1)
my_pal = {"4CM": "g", 
          "4EM": "m", 
          "4Naive":"#a4e0ef", 
          "Host":"#ff7f0e",
#             "donor":"blue", 
#           "host":"#DC582A",
         "4EM.CD69+":"#02feff",
         "Kihi":"#7d81fc",
         "Kilo":"#f50aff",
         "Kimid":"r"}

my_marks = {"4CM": "o",#"$\circ$", 
           "4EM": "^",
         "4Naive":"o",
         "4EM.CD69+CD103+":"P",
         "Old CHIM":"^"}

scatterkwags = {#"palette": my_pal,
                  #"markers":my_marks,
                    'edgecolor':"k",
                "alpha":0.7,
                "zorder":1
                
}

dfyfp = pd.read_csv('/opt/mesh/tiree/elise/samples_1/data_yfp_4.csv',index_col='MouseID')
dftom = pd.read_csv('/opt/mesh/tiree/elise/samples_1/data_tom_4.csv',index_col='MouseID')


def set_size(width, fraction=1, subplots=(1, 1)):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float or string
            Document width in points, or string of predined document type
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy
    subplots: array-like, optional
            The number of rows and columns of subplots.
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    if width == 'thesis':
        width_pt = 426.79135
    elif width == 'beamer':
        width_pt = 307.28987
    else:
        width_pt = width

    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)

def enumerated_product(*args):
    yield from zip(product(*(range(len(x)) for x in args)), product(*args))
    


In [2]:
#splines all data >5
# run = "_r6"
# location = '/opt/mesh/tiree/elise/samples_1/cd4_skin'+run

#splines remove >5 and <400
# run = "_r7"
# location = '/opt/mesh/tiree/elise/samples_1/cd4_skin'+run

#simuluatenous remove >5
# run = "_r1"
# location = '/opt/mesh/tiree/elise/samples_1/cd4_all'+run

#simuluatenous remove >5 and <400
# run = "_r2"
# location = '/opt/mesh/tiree/elise/samples_1/cd4_all'+run

#simuluatenous remove >5 tighter sigma prior
run = "_r3"
location = '/opt/mesh/tiree/elise/samples_1/cd4_all'+run


In [3]:

def arvizsetup_precursor(adhpop):
    fullloc = location+"/stan-cache-"+adhpop[0]+'4Naive'+adhpop[1]+adhpop[2]+"_"+str(1)+"/"
    
    fh = []
    
    #data slicing and manulipulation


    for f_name in os.listdir(fullloc):
        if f_name.endswith('.csv')&f_name.startswith('homog'):
            #print(f_name)
            fh.append(fullloc+f_name)

    sam1 = cmdstanpy.from_csv(fh)
        
    fullloc = location+"/stan-cache-"+adhpop[0]+'4EM'+adhpop[1]+adhpop[2]+"_"+str(1)+"/"
    
    fh = []
    
    #data slicing and manulipulation

    for f_name in os.listdir(fullloc):
        if f_name.endswith('.csv')&f_name.startswith('homog'):
            #print(f_name)
            fh.append(fullloc+f_name)

    sam2 = cmdstanpy.from_csv(fh)
    
    fullloc = location+"/stan-cache-"+adhpop[0]+'4CM'+adhpop[1]+adhpop[2]+"_"+str(1)+"/"
    
    fh = []
    
    #data slicing and manulipulation

    for f_name in os.listdir(fullloc):
        if f_name.endswith('.csv')&f_name.startswith('homog'):
            #print(f_name)
            fh.append(fullloc+f_name)

    sam3 = cmdstanpy.from_csv(fh)
    
    if adhpop[0]=='4EM.CD69+':
        
        fullloc = location+"/stan-cache-"+adhpop[0]+'69'+adhpop[1]+adhpop[2]+"_"+str(1)+"/"

        fh = []

        #data slicing and manulipulation

        for f_name in os.listdir(fullloc):
            if f_name.endswith('.csv')&f_name.startswith('homog'):
                #print(f_name)
                fh.append(fullloc+f_name)

        sam4 = cmdstanpy.from_csv(fh)

        ardata_1=arviz.from_cmdstanpy(#sam,
            posterior=sam1,
            # prior=sam_pB,
            # posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
            log_likelihood="log_lik",
        )

        ardata_2=arviz.from_cmdstanpy(#sam,
            posterior=sam2,
            # prior=sam_pL,
            # posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
            log_likelihood="log_lik",
        )
        ardata_3=arviz.from_cmdstanpy(#sam,
            posterior=sam3,
            # prior=sam_pB,
            # posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
            log_likelihood="log_lik",
        )

        ardata_4=arviz.from_cmdstanpy(#sam,
            posterior=sam4,
            # prior=sam_pL,
            # posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
            log_likelihood="log_lik",
        )


        compare_dict_linear = {"4Naive": ardata_1, "4EM": ardata_2, "4CM": ardata_3, "69": ardata_4}
    else:
        ardata_1=arviz.from_cmdstanpy(#sam,
            posterior=sam1,
            # prior=sam_pB,
            # posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
            log_likelihood="log_lik",
        )

        ardata_2=arviz.from_cmdstanpy(#sam,
            posterior=sam2,
            # prior=sam_pL,
            # posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
            log_likelihood="log_lik",
        )
        ardata_3=arviz.from_cmdstanpy(#sam,
            posterior=sam3,
            # prior=sam_pB,
            # posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
            log_likelihood="log_lik",
        )


        compare_dict_linear = {"4Naive": ardata_1, "4EM": ardata_2, "4CM": ardata_3}

    print(adhpop)
    results = (arviz.compare(compare_dict_linear,method='BB-pseudo-BMA'))
    results = results.drop(columns=['warning', 'scale'])
    print(results)
    results['population']=adhpop[0]
    results['ki']=adhpop[1]
    results['organ']=adhpop[2]

    results.to_csv('elpd_byprecursor.csv', mode='a', index=True, header=True)
    #arviz.plot_compare(results)
 

In [4]:

def arvizsetup_kimode(adhpop):
    fullloc = location+"/stan-cache-"+adhpop[0]+adhpop[1]+'Kihi'+adhpop[2]+"_"+str(1)+"/"
    
    fh = []
    
    #data slicing and manulipulation


    for f_name in os.listdir(fullloc):
        if f_name.endswith('.csv')&f_name.startswith('homog'):
            #print(f_name)
            fh.append(fullloc+f_name)

    sam1 = cmdstanpy.from_csv(fh)
        
    fullloc = location+"/stan-cache-"+adhpop[0]+adhpop[1]+'Kimid'+adhpop[2]+"_"+str(1)+"/"
    
    fh = []
    
    #data slicing and manulipulation

    for f_name in os.listdir(fullloc):
        if f_name.endswith('.csv')&f_name.startswith('homog'):
            #print(f_name)
            fh.append(fullloc+f_name)

    sam2 = cmdstanpy.from_csv(fh)
    
    fullloc = location+"/stan-cache-"+adhpop[0]+adhpop[1]+'Kilo'+adhpop[2]+"_"+str(1)+"/"
    
    fh = []
    
    #data slicing and manulipulation

    for f_name in os.listdir(fullloc):
        if f_name.endswith('.csv')&f_name.startswith('homog'):
            #print(f_name)
            fh.append(fullloc+f_name)

    sam3 = cmdstanpy.from_csv(fh)
    
    if (adhpop[0]=='4EM')&(adhpop[1]=='69'):
        print(adhpop)
    else:
        ardata_1=arviz.from_cmdstanpy(#sam,
                posterior=sam1,
                # prior=sam_pB,
                # posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
                log_likelihood="log_lik",
            )

        ardata_2=arviz.from_cmdstanpy(#sam,
                posterior=sam2,
                # prior=sam_pL,
                # posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
                log_likelihood="log_lik",
            )
        ardata_3=arviz.from_cmdstanpy(#sam,
                posterior=sam3,
                # prior=sam_pB,
                # posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
                log_likelihood="log_lik",
            )


        compare_dict_linear = {"KiHI": ardata_1, "KiMid": ardata_2, "KiLo": ardata_3}

        print(adhpop)
        results = (arviz.compare(compare_dict_linear,method='BB-pseudo-BMA'))
        results = results.drop(columns=['warning', 'scale'])
        print(results)
        results['population']=adhpop[0]
        results['precursor']=adhpop[1]
        results['organ']=adhpop[2]

        results.to_csv('elpd_bykimode.csv', mode='a', index=True, header=True)
        
    #arviz.plot_compare(results)
 

In [5]:

def processing(fullloc):
    fh = []
    
    #data slicing and manulipulation


    for f_name in os.listdir(fullloc):
        if f_name.endswith('.csv')&f_name.startswith('homog'):
            #print(f_name)
            fh.append(fullloc+f_name)
    sam = (cmdstanpy.from_csv(fh))
    
    ardata_1=arviz.from_cmdstanpy(#sam,
            posterior=sam,
            # prior=sam_pB,
            # posterior_predictive={"ppd_alpha_A","ppd_alpha_B","ppd_delta_A","ppd_delta_B","ppd_gamma","ppd_beta","ppd_Source","ppd_fs"},
            log_likelihood="log_lik",
        )
    
    return ardata_1

def arviz_full(adhpop):
    fullloc = location+"/stan-cache-"+adhpop[0]+'4CMKihi'+adhpop[1]+'_1'+"/"
    sam1 = processing(fullloc)
        
    fullloc = location+"/stan-cache-"+adhpop[0]+'4CMKilo'+adhpop[1]+'_1'+"/"
    sam2 = processing(fullloc)
    
    fullloc = location+"/stan-cache-"+adhpop[0]+'4CMKimid'+adhpop[1]+'_1'+"/"
    sam3 = processing(fullloc)
        
    fullloc = location+"/stan-cache-"+adhpop[0]+'4EMKihi'+adhpop[1]+'_1'+"/"
    sam4 = processing(fullloc)
    
    fullloc = location+"/stan-cache-"+adhpop[0]+'4EMKilo'+adhpop[1]+'_1'+"/"
    sam5 = processing(fullloc)
        
    fullloc = location+"/stan-cache-"+adhpop[0]+'4EMKimid'+adhpop[1]+'_1'+"/"
    sam6 = processing(fullloc)
    
    fullloc = location+"/stan-cache-"+adhpop[0]+'4NaiveKihi'+adhpop[1]+'_1'+"/"
    sam7 = processing(fullloc)
        
    fullloc = location+"/stan-cache-"+adhpop[0]+'4NaiveKilo'+adhpop[1]+'_1'+"/"
    sam8 = processing(fullloc)
    
    fullloc = location+"/stan-cache-"+adhpop[0]+'4NaiveKimid'+adhpop[1]+'_1'+"/"
    sam9 = processing(fullloc)
        
    fullloc = location+"/stan-cache-"+adhpop[0]+'69Kihi'+adhpop[1]+'_1'+"/"
    sam10 = processing(fullloc)
    
    fullloc = location+"/stan-cache-"+adhpop[0]+'69Kilo'+adhpop[1]+'_1'+"/"
    sam11 = processing(fullloc)
        
    fullloc =location+"/stan-cache-"+adhpop[0]+'69Kimid'+adhpop[1]+'_1'+"/"
    sam12 = processing(fullloc)
    
    
    if (adhpop[0]!='4EM.CD69+'):
        

        compare_dict_linear = {'4CMKihi': sam1, 
                               '4CMKilo': sam2, 
                               '4CMKimid': sam3,
                               '4EMKihi': sam4, 
                               '4EMKilo': sam5, 
                               '4EMKimid': sam6,
                               '4NaiveKihi': sam7, 
                               '4NaiveKilo': sam8, 
                               '4NaiveKimid': sam9
                              }
    else:
        compare_dict_linear = {'4CMKihi': sam1, 
                               '4CMKilo': sam2, 
                               '4CMKimid': sam3,
                               '4EMKihi': sam4, 
                               '4EMKilo': sam5, 
                               '4EMKimid': sam6,
                               '4NaiveKihi': sam7, 
                               '4NaiveKilo': sam8, 
                               '4NaiveKimid': sam9,
                               '69Kihi': sam10, 
                               '69Kilo': sam11, 
                               '69Kimid': sam12
                              }

    print(adhpop)
    results = (arviz.compare(compare_dict_linear,method='BB-pseudo-BMA'))
    results = results.drop(columns=['warning', 'scale'])
    print(results)
    results['population']=adhpop[0]
    results['organ']=adhpop[1]

    results.to_csv('elpd_bykimodeandprecursor.csv', mode='a', index=True, header=True)
        
    #arviz.plot_compare(results)
 

In [6]:

typei = "homog"


precursorc = ['4Naive','4EM','4CM','69']
populationc = ['4EM','4EM.CD69+']
ki67c = ['Kihi','Kimid','Kilo']
organc = ['SK','LP']

dfoldyfp = dfyfp[(dfyfp['organ'] == organc[0])&(dfyfp['Timepoint'] > 7)]
dfoldtom = dftom[(dftom['organ'] == organc[0])&(dftom['Days_post_treatment'] > 7)]

for idx, adhpop in enumerated_product(populationc,ki67c,organc):
    arvizsetup_precursor(adhpop)


('4EM', 'Kihi', 'SK')
        rank    elpd_loo     p_loo  elpd_diff        weight         se  \
4EM        0  -68.821972  8.213825   0.000000  9.921230e-01  10.257095   
4CM        1  -75.675877  8.634552   6.853906  7.876974e-03  10.354310   
4Naive     2 -104.488193  7.433533  35.666221  3.265230e-11   9.653351   

             dse  
4EM     0.000000  
4CM     2.216774  
4Naive  5.503303  
('4EM', 'Kihi', 'LP')
        rank    elpd_loo     p_loo  elpd_diff        weight         se  \
4EM        0  -90.417649  6.167265   0.000000  9.806064e-01  10.394068   
4CM        1  -96.577590  6.596633   6.159941  1.939356e-02  10.775355   
4Naive     2 -149.011882  5.908250  58.594232  1.343204e-19  10.837600   

             dse  
4EM     0.000000  
4CM     2.693104  
4Naive  8.838991  
('4EM', 'Kimid', 'SK')
        rank    elpd_loo     p_loo  elpd_diff        weight         se  \
4EM        0  -69.310596  7.679325   0.000000  9.888969e-01  10.269872   
4CM        1  -75.665059  7.877161   6.

In [7]:

typei = "homog"

precursorc = ['4Naive','4EM','4CM','69']
populationc = ['4EM','4EM.CD69+']
ki67c = ['Kihi','Kimid','Kilo']
organc = ['SK','LP']

dfoldyfp = dfyfp[(dfyfp['organ'] == organc[0])&(dfyfp['Timepoint'] > 7)]
dfoldtom = dftom[(dftom['organ'] == organc[0])&(dftom['Days_post_treatment'] > 7)]

for idx, adhpop in enumerated_product(populationc,precursorc,organc):
    arvizsetup_kimode(adhpop)


('4EM', '4Naive', 'SK')
       rank    elpd_loo     p_loo  elpd_diff    weight         se       dse
KiLo      0  -99.960441  7.592248   0.000000  0.527852  10.028770  0.000000
KiMid     1 -100.100052  7.276365   0.139611  0.459569   9.800040  0.173117
KiHI      2 -104.488193  7.433533   4.527751  0.012579   9.810783  1.358199
('4EM', '4Naive', 'LP')
       rank    elpd_loo     p_loo  elpd_diff    weight         se       dse
KiMid     0 -142.138687  6.380466   0.000000  0.538602  10.387568  0.000000
KiLo      1 -142.496490  6.781502   0.357803  0.383546  10.086563  0.158219
KiHI      2 -149.011882  5.908250   6.873194  0.077852  10.356115  5.389095
('4EM', '4EM', 'SK')
       rank   elpd_loo     p_loo  elpd_diff    weight        se       dse
KiHI      0 -68.821972  8.213825   0.000000  0.435044  9.788965  0.000000
KiLo      1 -69.199021  7.449321   0.377050  0.300559  9.884765  0.450884
KiMid     2 -69.310596  7.679325   0.488624  0.264397  9.906640  0.357053
('4EM', '4EM', 'LP')
      

In [8]:
typei = "homog"

precursorc = ['4Naive','4EM','4CM','69']
populationc = ['4EM','4EM.CD69+']
ki67c = ['Kihi','Kimid','Kilo']
organc = ['SK','LP']

dfoldyfp = dfyfp[(dfyfp['organ'] == organc[0])&(dfyfp['Timepoint'] > 7)]
dfoldtom = dftom[(dftom['organ'] == organc[0])&(dftom['Days_post_treatment'] > 7)]

for idx, adhpop in enumerated_product(populationc,organc):
    arviz_full(adhpop)


('4EM', 'SK')
             rank    elpd_loo     p_loo  elpd_diff        weight         se  \
4EMKihi         0  -68.821972  8.213825   0.000000  4.322517e-01  10.459595   
4EMKilo         1  -69.199021  7.449321   0.377050  2.977296e-01  10.361091   
4EMKimid        2  -69.310596  7.679325   0.488624  2.633216e-01  10.556341   
4CMKimid        3  -75.665059  7.877161   6.843087  3.107051e-03   9.817407   
4CMKihi         4  -75.675877  8.634552   6.853906  2.437690e-03   9.731407   
4CMKilo         5  -76.691231  8.401804   7.869259  1.152297e-03   9.828594   
4NaiveKilo      6  -99.960441  7.592248  31.138470  8.053932e-11  10.184033   
4NaiveKimid     7 -100.100052  7.276365  31.278081  6.427817e-11  10.331225   
4NaiveKihi      8 -104.488193  7.433533  35.666221  1.281922e-12  10.186288   

                  dse  
4EMKihi      0.000000  
4EMKilo      0.450884  
4EMKimid     0.357053  
4CMKimid     2.422905  
4CMKihi      2.216774  
4CMKilo      2.494477  
4NaiveKilo   5.623752  
4Na